# Example 3: Parse OW Feeds

In [1]:
import os
import sqlite3
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# Increase output width to better display the table of events
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [3]:
class HTMLTableParser:
   
    def parse_url(self, url, find_col_names = False):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        return [(table['id'],self.parse_html_table(table,find_col_names))\
                for table in soup.find_all('table')]  


    def parse_html_table(self, table, find_col_names):

        column_names = ['Event', 'AstNo', 'AstName', 'StarName', 'StarMag', 'Dur', 'MagDrop', 
                        'DTFrom', 'DTTo', 'SunDist', 'MoonDist', 'MoonIll', 'StarRA', 'StarDE', 
                        'Path', 'Details' ]

        n_columns = 0
        n_rows=0
        
        if find_col_names : column_names = [] 

        # Find number of rows and columns. Also try to find the column titles
        for row in table.find_all('tr'):
            
            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)
                    
            # Handle column names if we find them (and if find_col_names is True)
            if find_col_names:
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text(strip=True))

        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        
        df = pd.DataFrame(columns = columns, index= range(0,n_rows))
        
        row_marker = 0
        
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                href = ''
                for anchor in column.findAll('a'):
                    href = anchor['href']
                if href:
                    df.iat[row_marker,column_marker] = href
                else:
                    df.iat[row_marker,column_marker] = column.get_text(strip=True)
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1
                
        # Convert some columns to integer if possible
        for col in [ 'SunDist', 'MoonDist', 'MoonIll']:
            try:
                df[col] = df[col].astype(int)
            except ValueError:
                print("Step 1: Cannot convert")
                pass


        # Convert some columns to float if possible
        for col in [ 'StarMag', 'Dur', 'MagDrop']:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                print("Step 2: Cannot convert")                
                pass
        
        
        return df


In [6]:
# Integer in python/pandas becomes BLOB (binary) in sqlite
sqlite3.register_adapter(np.int64, int)
sqlite3.register_adapter(np.int32, int)

# Create SQLite Database
con = sqlite3.connect('owfeeds.sqlite')
cur = con.cursor()

table = 'feed_events'

cur.execute('DROP TABLE IF EXISTS %s' % table)

cur.execute("CREATE TABLE feed_events (Id INTEGER, DateUT TEXT, AstNo TEXT, AstName TEXT, StarName TEXT, \
            StarMag REAL, Dur REAL, MagDrop REAL, DTFrom TEXT, DTTo TEXT, SunDist INTEGER, MoonDist INTEGER, \
            MoonIll INTEGER, StarRA TEXT, StarDE TEXT, Path TEXT, Details TEXT, Feed TEXT)")

con.commit()


# Some OW feeds we want to parse

feeds = {
    'IBEROC'    : "http://ocultacions.astrosabadell.org/IBEROC/",
    'UKOCL'     : "http://ukoccultations.info/UKOCL/", 
    'Lucky-Star': "https://lesia.obspm.fr/lucky-star/feed/",
    'COMETOC'   : "http://ocultacions.astrosabadell.org/COMETOC/",
}

    
for feed, url in feeds.items():

    print(f"\nQuering feed : {feed}", end='')

    hp = HTMLTableParser()

    try:
        df = hp.parse_url(url)[0][1] # Grab the first table from the tuple

        print(f" => number of events = {len(df)}\n")

        df['Id'] = df.index + 1

        df['Date'] = '20' + df['Event'].str[0:2] + '-' + df['Event'].str[2:4] + '-' + df['Event'].str[4:6]

        #df['SSO'] = df['AstNo'] + ' ' + df['AstName']

        df['Path'] = url + df['Path']

        df['Details'] = url + df['Details']

        df['Feed'] = feed

        # Print the first entries
        #print(df.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,]].head())
        print(df.iloc[:,[0,1,2,3,4,5,6,9,10,11,12,13]].head())

        # Write events into SQLite Database
        try:
            cur.executemany("""INSERT INTO feed_events (
                            Id, DateUT, AstNo, AstName, StarName, StarMag, Dur, MagDrop,
                            DTFrom, DTTo, SunDist, MoonDist, MoonIll, StarRA, StarDE, Path, Details, Feed) 
                            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",
                            
                            list(df[['Id', 'Date', 'AstNo', 'AstName', 'StarName', 'StarMag', 'Dur', 'MagDrop', 
                                     'DTFrom', 'DTTo', 'SunDist', 'MoonDist', 'MoonIll', 'StarRA', 'StarDE', 
                                     'Path', 'Details', 'Feed']].to_records(
                            index=False,column_dtypes={ 'Id': 'object', 'Date' : 'object', 'AstNo': 'object', 'AstName': 'object',
                                           'StarName' : 'object', 'StarMag' : 'float', 'Dur' : 'float', 'MagDrop' : 'float', 
                                           'DTFrom' : 'object', 'DTTo' : 'object', 'SunDist' : 'int', 'MoonDist' : 'int', 
                                           'MoonIll' : 'int', 'StarRA' : 'object', 'StarDE' : 'object', 'Path': 'object', 
                                           'Details': 'object', 'Feed' : 'object'  })))

        except:
            pass

        con.commit()

    except KeyError:
        pass

# Finished


Quering feed : IBEROC => number of events = 5557

    Event  AstNo     AstName          StarName  StarMag  Dur  MagDrop  SunDist  MoonDist  MoonIll       StarRA       StarDE
0  200605  91981   1999 VB99  UCAC4 357-093325     13.8  0.7      4.4      172         5      100  17 24 33.40  -18 46 24.9
1  200605   3219      Komaki  TYC 6824-00255-1      9.2  1.6      6.1      173         7      100  17 16 17.60  -28 37 53.6
2  200605   2401     Aehlita  UCAC4 314-192526     14.2  0.8      2.7      160        18      100  18 22 38.78  -27 17 14.1
3  200605  53364   1999 JL77  UCAC4 407-068061     14.0  0.7      3.3      166        13      100  17  1 50.76  - 8 38 20.6
4  200605  67029  1999 XH166  UCAC4 292-121617     13.9  0.9      3.5      170        10      100  17 11 48.24  -31 46 34.1

Quering feed : UKOCL => number of events = 371

    Event  AstNo     AstName          StarName  StarMag  Dur  MagDrop  SunDist  MoonDist  MoonIll       StarRA       StarDE
0  200609  39405    Mosigkau  UC